In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install spark

     |████████████████████████████████| 41 kB 56 kB/s 
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58761 sha256=f923746faa809489a92869bd1a0fe272139a81ed189be1c7479f006afd55c70e
  Stored in directory: /root/.cache/pip/wheels/4e/0e/f1/164619f9920fb447d294afaae11a7715bd442ded7225953d72
Successfully built spark


In [ ]:
import spark

In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 73.0 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=0e26912003b796cee6b7cc90729e95a017224957c0730a45a585d66107001c25
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import QuantileDiscretizer

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark ML example on titanic data ") \
    .getOrCreate()

In [ ]:
s3_bucket_path = "/content/tripA_10-06-2021.csv"


In [ ]:
trip_df = spark.read.csv(s3_bucket_path,header = 'True',inferSchema='True')
trip_count = trip_df.count()
print(trip_count)


2074


In [ ]:
trip_df.show(15)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1|08/12/2018 08:49:...| 08/12/2018 09:00:...|              1|          2.2|         1|                 N|         234|         246|           1|       10.0|  0.5|    0.5|      2.25|         0.0|                  0.3|       13.55|
|       1|08/12/2018 08:49:...| 08/12/2018 09:00

In [ ]:

# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

In [ ]:
null_columns_count_list = null_value_count(trip_df)


In [ ]:
print(null_columns_count_list,trip_df)

[] DataFrame[VendorID: int, tpep_pickup_datetime: string, tpep_dropoff_datetime: string, passenger_count: int, trip_distance: double, RatecodeID: int, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: int, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double]


In [ ]:
mean_passenger_count = trip_df.select(mean('passenger_count')).collect()[0][0]
print(mean_passenger_count)

1.5274831243973


In [ ]:
trip_df.printSchema()


root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [ ]:
trip_df = trip_df.drop("tpep_pickup_datetime","tpep_dropoff_datetime","store_and_fwd_flag")


In [ ]:
trip_df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [ ]:
trip_df.show(5)

+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1|              1|          2.2|         1|         234|         246|           1|       10.0|  0.5|    0.5|      2.25|         0.0|                  0.3|       13.55|
|       1|              1|          2.2|         1|         234|         246|           1|       10.0|  0.5|    0.5|      2.25|         0.0|                  0.3|       13.55|
|       1|              1|          2.2|         1|         234|         246|           1|       10.0|  0.5|    0.5|    

In [ ]:
feature = VectorAssembler(inputCols=trip_df.columns[1:],outputCol="features")
feature_vector= feature.transform(trip_df)

In [ ]:
feature_vector.show(5)


+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|passenger_count|trip_distance|RatecodeID|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|            features|
+--------+---------------+-------------+----------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1|              1|          2.2|         1|         234|         246|           1|       10.0|  0.5|    0.5|      2.25|         0.0|                  0.3|       13.55|[1.0,2.2,1.0,234....|
|       1|              1|          2.2|         1|         234|         246|           1|       10.0|  0.5|    0.5|      2.25|         0.0|                  0.3|       13.55|[1.0,2.2,1.0,234....|
|       1|     

In [ ]:
(trainingData, testData) = feature_vector.randomSplit([0.8, 0.2],seed = 11)


In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(labelCol="VendorID", featuresCol="features")
#Training algo
lrModel = lr.fit(trainingData)
lr_prediction = lrModel.transform(testData)
lr_prediction.select("prediction", "VendorID", "features").show()
evaluator = MulticlassClassificationEvaluator(labelCol="VendorID", predictionCol="prediction", metricName="accuracy")

+----------+--------+--------------------+
|prediction|VendorID|            features|
+----------+--------+--------------------+
|       1.0|       1|[0.0,1.0,1.0,113....|
|       1.0|       1|[0.0,1.4,1.0,68.0...|
|       1.0|       1|[0.0,3.6,1.0,263....|
|       1.0|       1|[0.0,3.9,1.0,186....|
|       1.0|       1|[0.0,5.0,1.0,12.0...|
|       1.0|       1|[0.0,8.6,1.0,138....|
|       2.0|       1|[1.0,0.0,1.0,148....|
|       2.0|       1|[1.0,0.3,1.0,162....|
|       2.0|       1|[1.0,0.4,1.0,79.0...|
|       2.0|       1|[1.0,0.4,1.0,141....|
|       2.0|       1|[1.0,0.4,1.0,244....|
|       2.0|       1|[1.0,0.5,1.0,137....|
|       2.0|       1|[1.0,0.5,1.0,143....|
|       2.0|       1|[1.0,0.5,1.0,211....|
|       2.0|       1|[1.0,0.6,1.0,68.0...|
|       2.0|       1|[1.0,0.7,1.0,48.0...|
|       2.0|       1|[1.0,0.7,1.0,79.0...|
|       2.0|       1|[1.0,0.7,1.0,100....|
|       2.0|       1|[1.0,0.7,1.0,142....|
|       2.0|       1|[1.0,0.7,1.0,230....|
+----------

In [ ]:
lr_accuracy = evaluator.evaluate(lr_prediction)
print("Accuracy of LogisticRegression is = %g"% (lr_accuracy))
print("Test Error of LogisticRegression = %g " % (1.0 - lr_accuracy))

Accuracy of LogisticRegression is = 0.533654
Test Error of LogisticRegression = 0.466346 


In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol="VendorID", featuresCol="features")
dt_model = dt.fit(trainingData)
dt_prediction = dt_model.transform(testData)
dt_prediction.select("prediction", "VendorID", "features").show()

+----------+--------+--------------------+
|prediction|VendorID|            features|
+----------+--------+--------------------+
|       1.0|       1|[0.0,1.0,1.0,113....|
|       1.0|       1|[0.0,1.4,1.0,68.0...|
|       1.0|       1|[0.0,3.6,1.0,263....|
|       1.0|       1|[0.0,3.9,1.0,186....|
|       1.0|       1|[0.0,5.0,1.0,12.0...|
|       1.0|       1|[0.0,8.6,1.0,138....|
|       2.0|       1|[1.0,0.0,1.0,148....|
|       2.0|       1|[1.0,0.3,1.0,162....|
|       2.0|       1|[1.0,0.4,1.0,79.0...|
|       1.0|       1|[1.0,0.4,1.0,141....|
|       2.0|       1|[1.0,0.4,1.0,244....|
|       2.0|       1|[1.0,0.5,1.0,137....|
|       2.0|       1|[1.0,0.5,1.0,143....|
|       1.0|       1|[1.0,0.5,1.0,211....|
|       2.0|       1|[1.0,0.6,1.0,68.0...|
|       2.0|       1|[1.0,0.7,1.0,48.0...|
|       2.0|       1|[1.0,0.7,1.0,79.0...|
|       1.0|       1|[1.0,0.7,1.0,100....|
|       2.0|       1|[1.0,0.7,1.0,142....|
|       2.0|       1|[1.0,0.7,1.0,230....|
+----------

In [ ]:
dt_accuracy = evaluator.evaluate(dt_prediction)
print("Accuracy of DecisionTreeClassifier is = %g"% (dt_accuracy))
print("Test Error of DecisionTreeClassifier = %g " % (1.0 - dt_accuracy))

Accuracy of DecisionTreeClassifier is = 0.567308
Test Error of DecisionTreeClassifier = 0.432692 


In [ ]:

from pyspark.ml.classification import RandomForestClassifier
rf = DecisionTreeClassifier(labelCol="VendorID", featuresCol="features")
rf_model = rf.fit(trainingData)
rf_prediction = rf_model.transform(testData)
rf_prediction.select("prediction", "VendorID", "features").show()

+----------+--------+--------------------+
|prediction|VendorID|            features|
+----------+--------+--------------------+
|       1.0|       1|[0.0,1.0,1.0,113....|
|       1.0|       1|[0.0,1.4,1.0,68.0...|
|       1.0|       1|[0.0,3.6,1.0,263....|
|       1.0|       1|[0.0,3.9,1.0,186....|
|       1.0|       1|[0.0,5.0,1.0,12.0...|
|       1.0|       1|[0.0,8.6,1.0,138....|
|       2.0|       1|[1.0,0.0,1.0,148....|
|       2.0|       1|[1.0,0.3,1.0,162....|
|       2.0|       1|[1.0,0.4,1.0,79.0...|
|       1.0|       1|[1.0,0.4,1.0,141....|
|       2.0|       1|[1.0,0.4,1.0,244....|
|       2.0|       1|[1.0,0.5,1.0,137....|
|       2.0|       1|[1.0,0.5,1.0,143....|
|       1.0|       1|[1.0,0.5,1.0,211....|
|       2.0|       1|[1.0,0.6,1.0,68.0...|
|       2.0|       1|[1.0,0.7,1.0,48.0...|
|       2.0|       1|[1.0,0.7,1.0,79.0...|
|       1.0|       1|[1.0,0.7,1.0,100....|
|       2.0|       1|[1.0,0.7,1.0,142....|
|       2.0|       1|[1.0,0.7,1.0,230....|
+----------

In [ ]:

rf_accuracy = evaluator.evaluate(rf_prediction)
print("Accuracy of RandomForestClassifier is = %g"% (rf_accuracy))
print("Test Error of RandomForestClassifier  = %g " % (1.0 - rf_accuracy))

Accuracy of RandomForestClassifier is = 0.567308
Test Error of RandomForestClassifier  = 0.432692 
